In [2]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

ROOT = os.path.dirname(os.path.abspath(""))
ROOT

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import os
import json
from collections import defaultdict
from tqdm import tqdm

# Initialize a defaultdict for storing lists of pages for each document.
documents = defaultdict(list)

# Iterate over the list of files in the "ocr" directory.
for filename in tqdm(os.listdir("ocr")):
    # Only process .json files.
    if filename.endswith(".json"):
        # Extract the document ID and the page number.
        doc_id, page_num = filename.split('_p')
        page_num = int(page_num[:-5])  # Remove the '.json' and convert to an integer.

        # Append the page filename to the document's list, along with the page number.
        documents[doc_id].append((page_num, filename))

# Limit to only the first 5 documents.
ocr_dX = []
for i, (doc_id, pages) in enumerate(documents.items()):
    if i >= 5:  # Stop after processing 5 documents.
        break

    # Sort the pages by page number and load the JSON content.
    sorted_pages = [json.load(open(f"ocr/{file}", 'r')) for _, file in sorted(pages)]
    
    # Append the sorted pages to the main list.
    ocr_dX.append(sorted_pages)


100%|██████████| 64057/64057 [00:00<00:00, 780529.17it/s]


In [16]:
for d, doc in enumerate(ocr_dX):
	for p, page in enumerate(doc):
		if not page.get("LINE"):
			continue
		page_text = " ".join([line["Text"] for line in page["LINE"]])
		# save page_text to a .txt file in documents_text folder
		with open(f"documents_text/{d}_{p}.txt", "w") as f:
			f.write(page_text)

---

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
	# embed_model="local"
)

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./documents_text").load_data()

index = VectorStoreIndex.from_documents(documents)

In [11]:
query_engine = index.as_query_engine()

response = query_engine.query("query string")
# Error solution here: https://stackoverflow.com/questions/76771761/why-does-llama-index-still-require-an-openai-key-when-using-hugging-face-local-e
# But is for deprecated, now doesn't work2/

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******